# V3 Auto Finance Table

In [24]:
# Import Libraries
########################################
#         Data Import from Sheets      #
########################################
import gspread
import pandas as pd
from datetime import datetime, timedelta
import calendar
import matplotlib.pyplot as plt
from oauth2client.service_account import ServiceAccountCredentials
import smtplib
from email.message import EmailMessage
import os
import mimetypes
from pathlib import Path
import json
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# Potential Future Use
# email_username = os.getenv("EMAIL_USERNAME")
# email_password = os.getenv("EMAIL_PASSWORD")


In [25]:
# Credentials for Importing Google Sheets Data
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
# creds = ServiceAccountCredentials.from_json_keyfile_name(os.getenv('TRY'), scope)
json_keyfile_dict = json.loads(os.getenv("JSON_KEYFILE_DICT"))
creds = ServiceAccountCredentials.from_json_keyfile_dict(json_keyfile_dict, scope)
client = gspread.authorize(creds)
spreadsheet_key = os.getenv("SPREADSHEET_KEY2")
spreadsheet = client.open_by_key(spreadsheet_key)

In [26]:
# Import Expenses Data
sheet_title = 'Transactions'  
sheet = spreadsheet.worksheet(sheet_title)
expenses_data = sheet.get_all_records()
expenses_data = pd.DataFrame(expenses_data)

# Clean Time data
expenses_data["Date"] = expenses_data["Date"].astype(str).str.strip()

expenses_data["Date"] = pd.to_datetime(
    expenses_data["Date"],
    format="mixed",            
    utc=True,                  
    dayfirst=False          
)

expenses_data["Date"] = expenses_data["Date"].dt.tz_convert(None)

# Remove Money Hold if applicable
expenses_data = expenses_data[expenses_data['Category'] != "Hold"]

# expenses_data["Date"] = pd.to_datetime(expenses_data['Date'])

In [27]:
# Import Income Data
sheet_title = 'Income'  
sheet = spreadsheet.worksheet(sheet_title)
income_data = sheet.get_all_records()
income_data = pd.DataFrame(income_data)

# Clean Time data
income_data["Date"] = income_data["Date"].astype(str).str.strip()

income_data["Date"] = pd.to_datetime(
    income_data["Date"],
    format="mixed",            
    utc=True,                  
    dayfirst=False          
)

income_data["Date"] = income_data["Date"].dt.tz_convert(None)

In [ ]:
df = expenses_data
df = df.rename(columns={'Amount': 'expenses'})
df['Year-Month'] = df['Date'].dt.to_period('M')

# Group by 'Year-Month'
monthly_expenses_all = df.groupby('Year-Month')['expenses'].sum().round(2).reset_index()

# Filter Outliers
df2 = expenses_data[expenses_data['Outlier'] != 1]
df2 = df2.rename(columns={'Amount': 'expenses_no_outlier'})
df2['Year-Month'] = df2['Date'].dt.to_period('M')
monthly_expenses_no_outlier = df2.groupby('Year-Month')['expenses_no_outier'].sum().round(2).reset_index()

table = pd.merge(monthly_expenses_no_outlier, monthly_expenses_all, on='Year-Month')

In [29]:
# Calculate Amount Spent, Amount Earned, Delta Table Cummulative Sum Per Month
df3 = income_data
df3 = df3.rename(columns={'Amount': 'income'})
df3['Year-Month'] = df3['Date'].dt.to_period('M')

# Group by 'Year-Month'
monthly_income_all = df3.groupby('Year-Month')['income'].sum().round(2).reset_index()
monthly_income_all

table = pd.merge(table, monthly_income_all, on='Year-Month', how="left")
table = table.fillna(0)

In [31]:
table['plus_minus'] = (table['income'] - table['expenses']).round(2)
table['net_worth'] = table['plus_minus'].cumsum().round(2)

In [ ]:
df4 = pd.DataFrame({
    "Year-Month": ["AVERAGE"],
    "expenses_no_outlier": [table['expenses_no_outlier'].mean().round(2)],
    "expenses": [table['expenses'].mean().round(2)],
    "income": [table['income'].mean().round(2)],
    "plus_minus": [table['plus_minus'].mean().round(2)]
})

table = pd.concat([table, df4])

In [38]:
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


# 2. Convert to HTML with inline CSS for gridlines
html_table = table.to_html(index=False, border=0)
styled_html = f"""
<html>
  <head>
    <style>
      table, th, td {{
        border: 1px solid black;
        border-collapse: collapse;
        padding: 4px;
      }}
      th {{ background-color: #f2f2f2; }}
    </style>
  </head>
  <body>
    <p>Hi there,</p>
    <p>See the table below:</p>
    {html_table}
    <p>Best,<br>Your Python Script</p>
  </body>
</html>
"""

# 3. Build the email message
msg = MIMEMultipart('alternative')
msg['Subject'] = "Monthly Expenses Overall"
msg['From']    = "lorossiprojects@gmail.com"
msg['To']      = "lohan.rossi@hotmail.com"

# Attach the HTML part
msg.attach(MIMEText(styled_html, 'html'))

# 4. Send via SMTP (example: Gmail SMTP server)
smtp_server = "smtp.gmail.com"
smtp_port   = 587
smtp_user   = "lorossiprojects@gmail.com"
smtp_pass   = "esso ebao dfml ccwj"

with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.ehlo()
    server.starttls()
    server.login(smtp_user, smtp_pass)
    server.send_message(msg)

print("Email sent successfully!")


Email sent successfully!
